### read_csv

file, sep , header, names , index_col, usecols, dtyes, parse_dates
- add duplicate suffix to columns
- header = None
- header = 0 , skip first row cols = [] , pass the colname 

### setting an index 
- parse_dates=True
- usecols , select the columns , saves processing time and memory ( add index also otherwise error)

### select rows 
nrows 
skiprows based on index position, nrows works after skiprows
- skiprows = lambda x:x%2 == 1

### missing values 
- na_values =[".","Missing"] --> NaN

### parsing dates parse_dates=["Date"],
infer_datetime_format=True

### dtype=({"Sales":"Int32"})

In [1]:
import pandas as pd 

In [11]:
columns = ["index", "Date", "Oil Price" , "Date2"]
string_slice = lambda x:int(x[-2:])

oil = pd.read_csv("./retail/oil.csv",
                  header = 0,
                # names = columns,
               #  parse_dates=["Date", "Date2"],
                # infer_datetime_format=True
                # index_col = "Date",
                  # dtype ={"index":"Int16"},
                 # converters = {"Date2":string_slice}
                 )
oil.rename(columns={"dcoilwtico": "Oil Price"}, inplace=True)
oil["index"] = oil.index

oil.head()

,date,Oil Price,index
0,2013-01-01,NaN,0
1,2013-01-02,93.14,1
2,2013-01-03,92.97,2
3,2013-01-04,93.12,3
4,2013-01-07,93.20,4


### converters 
currency = lambda x:f"${x}.00"
missing = lambda x:x if x not in ["MISSING","."] else 0

pd.read_csv(
"monthly_sales.csv",
converters={"Sales":missing,
"Sales.1":currency
}
)

### Exercise 1 streamlined transaction 

In [23]:
columns = ["Date", "Store_Number", "Transaction_Count" ]
transactions = pd.read_csv(
                          "./retail/transactions.csv",
                          header=0,
                          names=columns, #names = ["Date", "Store_Number", "Transaction_Count" ]
                          skiprows=[0],
                          parse_dates=["Date"],
                          dtype = {"Store_Number":"Int8","Transaction_Count":"Int16"}).assign(
                              target_pct= lambda x:(x["Transaction_Count"]/2500),
                              met_target= lambda x:(x["Transaction_Count"]/2500 >= 1),
                              bonus_payable = lambda x:(x["Transaction_Count"]/2500 >= 1 * 100),
                              month = lambda x:x["Date"].dt.month,
                              day_of_week = lambda x: x["Date"].dt.dayofweek,
                                                    ).astype({"target_pct" :"Float32",
                                                             "month":"Int8",
                                                             "day_of_week":"Int8"}
                                                            )

transactions.head()

,Date,Store_Number,Transaction_Count,target_pct,met_target,bonus_payable,month,day_of_week
0,2013-01-02,1,2111,0.8444,False,False,1,2
1,2013-01-02,2,2358,0.9432,False,False,1,2
2,2013-01-02,3,3487,1.3948,True,False,1,2
3,2013-01-02,4,1922,0.7688,False,False,1,2
4,2013-01-02,5,1903,0.7612,False,False,1,2


In [25]:
transactions.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83487 entries, 0 to 83486
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               83487 non-null  datetime64[ns]
 1   Store_Number       83487 non-null  Int8          
 2   Transaction_Count  83487 non-null  Int16         
 3   target_pct         83487 non-null  Float32       
 4   met_target         83487 non-null  boolean       
 5   bonus_payable      83487 non-null  boolean       
 6   month              83487 non-null  Int8          
 7   day_of_week        83487 non-null  Int8          
dtypes: Float32(1), Int16(1), Int8(3), boolean(2), datetime64[ns](1)
memory usage: 2.1 MB


### reading txt files 
pd.read_csv('tab_seperated.txt', sep='\t')
pd.read_excel('monthly_sales.xlsx', sheet_name=1) # 0 index or refer sheet name 
all_cells = pd.concat(pd.read_excel('monthly_sales.xlsx', sheet_name=None),
ignore_index=True
)

### export dataframes 
to_csv() to_excel()

my_df.to_csv("cleaned_data.csv")
my_df.to_csv("cleaned_data.txt, sep=\t")
my_df.to_excel("cleaned_data.xlsx",sheet_name="October_Sales")

with pd.ExcelWriter("cleaned_data.xlsx") as writer:
   my_df.to_excel(writer,sheet_name="October_Sales")
   my_df.to_excel(writer,sheet_name="November_Sales")

In [26]:
transactions.head()

,Date,Store_Number,Transaction_Count,target_pct,met_target,bonus_payable,month,day_of_week
0,2013-01-02,1,2111,0.8444,False,False,1,2
1,2013-01-02,2,2358,0.9432,False,False,1,2
2,2013-01-02,3,3487,1.3948,True,False,1,2
3,2013-01-02,4,1922,0.7688,False,False,1,2
4,2013-01-02,5,1903,0.7612,False,False,1,2


In [30]:
transactions.dtypes

Date                 datetime64[ns]
Store_Number                   Int8
Transaction_Count             Int16
target_pct                  Float32
met_target                  boolean
bonus_payable               boolean
month                          Int8
day_of_week                    Int8
dtype: object

In [32]:
transactions["year"] = transactions["Date"].dt.year

transactions.head()

,Date,Store_Number,Transaction_Count,target_pct,met_target,bonus_payable,month,day_of_week,year
0,2013-01-02,1,2111,0.8444,False,False,1,2,2013
1,2013-01-02,2,2358,0.9432,False,False,1,2,2013
2,2013-01-02,3,3487,1.3948,True,False,1,2,2013
3,2013-01-02,4,1922,0.7688,False,False,1,2,2013
4,2013-01-02,5,1903,0.7612,False,False,1,2,2013


In [41]:
with pd.ExcelWriter("TransactionYear.xlsx") as writer:
    for year in transactions["year"].unique():
        transactions.loc[transactions["Date"].dt.year == year].to_excel(writer,sheet_name=str(year)) 

In [42]:
for year in transactions["year"].unique():
    transactions.loc[transactions["Date"].dt.year == year].to_csv(
        f"transactions_{year}.csv"
    )